# Welcome to WQD7003 Data Analytics Lab
This code is generated for the purpose of WQD7003 module.

Created by Shier Nee Saw

Reference: Python for Data Analysis O'Reily

# Hierarchical Indexing

Hierarchical indexing is an important feature of pandas enabling you to have multiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form. Let’s start with a simple example; create a Series with a list of lists or arrays as the index:

In [59]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(10),
                 index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                 [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])

data

a  1    0.608970
   2   -1.270276
   3    1.925750
b  1    0.760329
   2    0.244607
   3   -0.603453
c  1    0.019586
   2    0.226678
d  2    0.042121
   3   -0.706121
dtype: float64

In [60]:
# What you’re seeing is a prettified view of a Series with a MultiIndex as its index. The “gaps” in the index display mean “use the label directly above”:

data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [61]:
# With a hierarchically-indexed object, so-called partial indexing is possible, enabling
# you to concisely select subsets of the data:

# return values with index b
data['b']

1    0.760329
2    0.244607
3   -0.603453
dtype: float64

In [62]:
# return values with index b until d
data['b':'d']

b  1    0.760329
   2    0.244607
   3   -0.603453
c  1    0.019586
   2    0.226678
d  2    0.042121
   3   -0.706121
dtype: float64

In [63]:
# return values with index b and d
data.loc[['b', 'd']]

b  1    0.760329
   2    0.244607
   3   -0.603453
d  2    0.042121
   3   -0.706121
dtype: float64

In [64]:
# Selection is even possible in some cases from an “inner” level:
# return all values with index 2. You need a ':' to indicate all values in the first level

data[:, 2]

a   -1.270276
b    0.244607
c    0.226678
d    0.042121
dtype: float64

Hierarchical indexing plays a critical role in reshaping data and group-based operations
like forming a pivot table. For example, this data could be rearranged into a DataFrame
using its unstack method:

In [65]:
data.unstack()

,1,2,3
a,0.608970,-1.270276,1.925750
b,0.760329,0.244607,-0.603453
c,0.019586,0.226678,NaN
d,NaN,0.042121,-0.706121


In [66]:
# The inverse operation of unstack is stack

data.unstack().stack()

a  1    0.608970
   2   -1.270276
   3    1.925750
b  1    0.760329
   2    0.244607
   3   -0.603453
c  1    0.019586
   2    0.226678
d  2    0.042121
   3   -0.706121
dtype: float64

In [67]:
# With a DataFrame, either axis can have a hierarchical index

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [68]:
# The hierarchical levels can have names (as strings or any Python objects). If so, these
# will show up in the console output (don’t confuse the index names with the axis labels!):

frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [69]:
# With partial column indexing you can similarly select groups of columns:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

# Data Wrangling: Clean, Transform, Merge, Reshape

Much of the programming work in data analysis and modeling is spent on data preparation: loading, cleaning, transforming, and rearranging.

Sometimes the way that data is stored in files or databases is not the way you need it for a data processing application.

Many people choose to do ad hoc processing of data from one form to another using a general purpose programming, like Python, Perl, R, or Java, or UNIX text processing tools like sed or awk.

Fortunately, pandas along with the Python standard library provide you with a high-level, flexible, and high-performance set of core manipulations and algorithms to enable you to wrangle data into the right form without much trouble.
  

## Combining and Merging Data Sets

*  pandas.merge - connects rows in DataFrames based on one or more keys. This will be familiar to users of SQL or other relational databases, as it implements database join operations.
*  pandas.concat - glues or stacks together objects along an axis.

Merge or join operations combine data sets by linking rows using one or more keys. These operations are central to relational databases. The merge function in pandas is the main entry point for using these algorithms on your data


In [70]:
import pandas as pd

df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

print(df1)
print('----')
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
----
  key  data2
0   a      0
1   b      1
2   d      2


In [71]:
# This is an example of a many-to-one merge situation; the data in df1 has multiple rows
# labeled a and b, whereas df2 has only one row for each value in the key column. Calling
# merge with these objects we obtain:
# reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

pd.merge(df1, df2)


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Note that I didn’t specify which column to join on. If not specified, merge uses the overlapping column names as the keys.

It’s a good practice to specify explicitly.

In [72]:
pd.merge(df1, df2, on='key')


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [73]:
# If the column names are different in each object, you can specify them separately:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

print(df3)
print('----')
print(df4)



  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
----
  rkey  data2
0    a      0
1    b      1
2    d      2


In [74]:
# Merge df1 and df2 on the lkey and rkey columns.

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [75]:
# outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

df1 = pd.DataFrame({'a': ['foo', 'bar'], 'b': [1, 2]})
df2 = pd.DataFrame({'a': ['foo', 'baz'], 'c': [3, 4]})

pd.merge(df1, df2, how='outer', on='a')

,a,b,c
0,foo,1.0,3.0
1,bar,2.0,NaN
2,baz,NaN,4.0


In [76]:
# inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

pd.merge(df1, df2, how='inner', on='a')

,a,b,c
0,foo,1,3


## Merging on Index

In some cases, the merge key or keys in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key:

In [77]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

print(left1)
print('---')
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
---
   group_val
a        3.5
b        7.0


In [78]:
# right_index = Use the index from the right DataFrame as the join key
# merge left1 and right1 using key. use the index from right1 as the join key

pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the union of them with an outer join:

In [79]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


## Concatenating Along an Axis

Another kind of data combination operation is alternatively referred to as concatenation, binding, or stacking. NumPy has a concatenate function for doing this with raw NumPy arrays:

In [80]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [81]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [83]:
# concat applies in pandas too

s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

print(s1)
print('--')
print(s2)
print('--')
print(s3)

a    0
b    1
dtype: int64
--
c    2
d    3
e    4
dtype: int64
--
f    5
g    6
dtype: int64


In [84]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [85]:
# If you pass axis=1, the result will instead be a DataFrame (axis=1 is the columns):
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [92]:
# In this case there is no overlap on the other axis, which as you can see is the sorted
# union (the 'outer' join) of the indexes. You can instead intersect them by passing
# join='inner'

s4 = pd.concat([s1 * 5, s3])

print(s1)
print('--')
print(s3)
print('--')
print(s4)

a    0
b    1
dtype: int64
--
f    5
g    6
dtype: int64
--
a    0
b    5
f    5
g    6
dtype: int64


In [90]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [93]:
# concat only the intersection
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [98]:
# Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this,
# use the keys argument:

result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [99]:
# you can unstack it
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [102]:
# The same logic extends to DataFrame objects

df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

print(df1)
print('---')
print(df2)
print('----')
pd.concat([df1, df2], axis=1)

   one  two
a    0    1
b    2    3
c    4    5
---
   three  four
a      5     6
c      7     8
----


,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [103]:
# A last consideration concerns DataFrames in which the row index is not meaningful in
# the context of the analysis:

df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

print(df1)
print('---')
print(df2)
print('----')
pd.concat([df1, df2], ignore_index=True) # notice that the index is ignore

          a         b         c         d
0 -1.497522 -0.703587  0.344305 -0.228678
1  0.367036 -1.508630  0.031554 -0.993914
2  1.788999  0.585140 -0.665198 -0.451383
---
          b         d         a
0 -1.466807 -1.269011  0.726198
1  0.468642  0.411967  0.589961
----


,a,b,c,d
0,-1.497522,-0.703587,0.344305,-0.228678
1,0.367036,-1.508630,0.031554,-0.993914
2,1.788999,0.585140,-0.665198,-0.451383
3,0.726198,-1.466807,NaN,-1.269011
4,0.589961,0.468642,NaN,0.411967


In [104]:
pd.concat([df1, df2], ignore_index=False) # notice that the index remains

,a,b,c,d
0,-1.497522,-0.703587,0.344305,-0.228678
1,0.367036,-1.508630,0.031554,-0.993914
2,1.788999,0.585140,-0.665198,-0.451383
0,0.726198,-1.466807,NaN,-1.269011
1,0.589961,0.468642,NaN,0.411967


# Data Transformation
## Removing duplicates



In [108]:
data = pd.DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [109]:
# The DataFrame method duplicated returns a boolean Series indicating whether each
# row is a duplicate or not:

data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [110]:
# Relatedly, drop_duplicates returns a DataFrame where the duplicated array is True
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [119]:
# Both of these methods by default consider all of the columns; alternatively you can
# specify any subset of them to detect duplicates. Suppose we had an additional column
# of values and wanted to filter duplicates only based on the 'k1' column:

data['v1'] = range(7)
print(data)

data.drop_duplicates(['k1'])

    k1  k2  v1
0  one   1   0
1  one   1   1
2  one   2   2
3  two   3   3
4  two   3   4
5  two   4   5
6  two   4   6


,k1,k2,v1
0,one,1,0
3,two,3,3


In [121]:
# duplicated and drop_duplicates by default keep the first observed value combination.
# Passing take_last=True will return the last one:

data.drop_duplicates(['k1'], keep='last')

,k1,k2,v1
2,one,2,2
6,two,4,6


In [124]:
# ccheck column k1 and k2 for duplicates, keep the first one.

data.drop_duplicates(['k1', 'k2'], keep='first')

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


## Transforming Data Using a Function or Mapping

For many data sets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about some kinds of meat:

In [125]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                              'corned beef', 'Bacon', 'pastrami', 'honey ham',
                              'nova lox'],
                   'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [127]:
# Suppose you wanted to add a column indicating the type of animal that each food came
# from. Let’s write down a mapping of each distinct meat type to the kind of animal:

meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}


In [131]:
# The map method on a Series accepts a function or dict-like object containing a mapping,
# but here we have a small problem in that some of the meats above are capitalized and
# others are not. Thus, we also need to convert each value to lower case:


# first map to lower case, then map into the respective animal
data['animal'] = data['food'].map(str.lower)
data

,food,ounces,animal
0,bacon,4.0,bacon
1,pulled pork,3.0,pulled pork
2,bacon,12.0,bacon
3,Pastrami,6.0,pastrami
4,corned beef,7.5,corned beef
5,Bacon,8.0,bacon
6,pastrami,3.0,pastrami
7,honey ham,5.0,honey ham
8,nova lox,6.0,nova lox


In [132]:
data['animal']=data['animal'].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [134]:
# or you can combine both of the mapping into one line

data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


## Replaceing values

Filling in missing data with the fillna method can be thought of as a special case of more general value replacement. While map, as you’ve seen above, can be used to modify a subset of values in an object, replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [136]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [137]:
# replace -999 with NaN

data.replace(-999, np.nan)


0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [138]:
# If you want to replace multiple values at once, you instead pass a list then the substitute
# value:

data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [139]:
# To use a different replacement for each value, pass a list of substitutes:

data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [140]:
# The argument passed can also be a dict:

data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [147]:
np.random.seed(12345) # set the seed for reproducibility

# create 1000 x 4 random number
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [148]:
# Suppose you wanted to find values in one of the columns exceeding three in magnitude:
# here, we use the column '3'

col = data[3]
col[np.abs(col) >3]

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [163]:
# To select all rows having a value exceeding 3 or -3,
# you can use the 'any' method on a boolean DataFrame

data[(np.abs(data) > 3).any(axis=1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [164]:
# To select all rows having values in all column exceeding 1 or -1,
# you can use the 'all' method on a boolean DataFrame

data[(np.abs(data) > 1).all(axis=1)]

,0,1,2,3
131,1.297622,-1.686933,1.089539,2.060882
398,-1.115689,1.234297,-1.893094,-1.661605
427,-1.276761,1.881156,1.108227,-1.751994
565,1.502867,-1.400992,1.105131,-1.123506
620,-1.026410,1.097896,-2.216861,-1.681929
639,-1.198140,-1.027268,-1.525054,-1.955038
697,1.190274,1.800285,-2.207419,-2.651879
737,1.101240,1.134073,1.073032,-1.073461
772,1.178483,1.063616,1.229272,-1.077152
857,1.522475,1.098180,1.116874,-1.591605


## Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix.

If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame containing k columns containing all 1’s and 0’s. pandas has a get_dummies function for doing this, though devising one yourself is not difficult.

In [165]:
# we have a categorical data in column key
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})

df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [166]:
# convert categorical data into dummy data
# you notice if the value is a, it is true in the column a while false in column b and c
# similary, if the value is b, it is true in the column b while false in column a and c

pd.get_dummies(df['key'])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In [168]:
# In some cases, you may want to add a prefix to the columns in the indicator DataFrame,
# which can then be merged with the other data. get_dummies has a prefix argument for
# doing just this:

dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)

df_with_dummy

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


In [173]:
# let's try with movie.dat dataset
# download movie.dat from the ODL platform
# upload to Google Colab: Refer to https://www.youtube.com/watch?v=I9zT-dC4Lw8&ab_channel=Dr.Vipin%27sClassroom
# after you have upload the file, run this cell

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('movies.dat', sep='::', header=None, names=mnames, encoding='latin1')
movies

<ipython-input-173-796064522ed1>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('movies.dat', sep='::', header=None, names=mnames, encoding='latin1')


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [176]:
# First, we extract the list of unique genres in the dataset (using a nice set.union trick):

genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [178]:
# Now, one way to construct the indicator DataFrame is to start with a DataFrame of all
# zeros:

dummies = pd.DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [184]:
# Now, iterate through each movie and set entries in each row of dummies to 1:

for i, gen in enumerate(movies.genres):
    col = gen.split('|')  # to split the genres if more than 1 genre
    dummies.loc[i, col] = 1



In [185]:
# Then, as above, you can combine this with movies:
# You will noticed that for the first instance
# Toy Story (1995) movies genres are animation, children and comedy.
# And the column of the animation, children and comedy will be equal to 1, the rest equals to 0

movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3879,3949,Requiem for a Dream (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3880,3950,Tigerland (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3881,3951,Two Family House (2000),Drama,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## String manipulation

Python has long been a popular data munging language in part due to its ease-of-use for string and text processing.

Most text operations are made simple with the string
object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by enabling you to apply
string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.

In [186]:
# In many string munging and scripting applications, built-in string methods are sufficient.
# As an example, a comma-separated string can be broken into pieces with split:

val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

In [188]:
# split is often combined with strip to trim whitespace (including newlines):
# notice the white space infront of ' guido' is removed

pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [189]:
# These substrings could be concatenated together with a two-colon delimiter using addition:
first, second, third = pieces
first + '::' + second + '::' + third


'a::b::guido'

In [190]:
# But, this isn’t a practical generic method. A faster and more Pythonic way is to pass a
# list or tuple to the join method on the string '::':

'::'.join(pieces)

'a::b::guido'

In [193]:
# Other methods are concerned with locating substrings. Using Python’s in keyword is
# the best way to detect a substring, though index and find can also be used:

# check if 'guido' in val
print(val)

'guido' in val

a,b, guido


True

In [194]:
# check the index of comma in val
# comma exist in the first index. Note: Python's index starts at 0

print(val)
val.index(',')

a,b, guido


1

In [197]:
# you can use 'find' to locate the index

val.find(',')

1

In [195]:
# if the string is not found, using index will raise ValueError

val.index(':')

ValueError: substring not found

In [196]:
# if the string is not found, using find will return -1

val.find(':')

-1

In [199]:
# Relatedly, count returns the number of occurrences of a particular substring

# count how many comma in val
print(val)
val.count(',')

a,b, guido


2

## Vectorized string functions in pandas

Cleaning up a messy data set for analysis often requires a lot of string munging and regularization. To complicate matters, a column containing strings will sometimes have
missing data:

In [206]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [207]:
# Check whether each email address has 'gmail' in it with str.contains

data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [208]:
# Using findall on the text produces a list of the e-mail addresses
# We notice the email addresses consists of 3 components: username, domain name, and domain suffix.
# We can segment each address around the parts of the pattern to segment:

pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

# Exercise: USDA Food Database

The US Department of Agriculture makes available a database of food nutrient information.

Ashley Williams, an English hacker, has made available a version of this database in JSON format (http://ashleyw.co.uk/project/food-nutrient-database).

In [258]:
# download foods-2011-10-03.json from the ODL platform
# upload to Google Colab: Refer to https://www.youtube.com/watch?v=I9zT-dC4Lw8&ab_channel=Dr.Vipin%27sClassroom
# after you have upload the file, run this cell

import json
db = json.load(open('foods-2011-10-03.json'))

nutrients = []
for rec in db:
  fnuts = pd.DataFrame(rec['nutrients'])
  fnuts['id'] = rec['id']
  nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

info_keys = ['description', 'group', 'id', 'manufacturer']
info = pd.DataFrame(db, columns=info_keys)


## Tasks
1. Check how many duplicated instance in *nutrients* dataframe. -- Ans: 14179
2. Drop the duplicates in *nutrients* dataframe -- Ans: After drop, you left 375176 rows
3. Rename column name of description with food, and group with fgroup for *info* dataframe
4. Rename column name of description with nutrient, and group with nutgroup for *nutrients* dataframe
4. merge *nutrient* and *info* dataframes by *id*
5. Plot the median Zinc values by nutrient group. Hint: use groupby yand .plot(kind='barh')


In [ ]:
# Your solution here

### Submission: File > Print > As PDF > Submit in ODL Platform
### Make sure the answer is visible in PDF format.
### Deadline: 1 week after today class.